In [1]:
 # Dependencies
from bs4 import BeautifulSoup
import requests
import os
import pymongo
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd
from zipfile import ZipFile
from IPython.display import display_html
import json
import pprint

In [13]:
# set up the splinter method
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [14]:
# define the target URLs
url1 = 'http://wdi.worldbank.org/table/6.4' # direction of trade, from low/high to high/low income economies
url2 = 'http://wdi.worldbank.org/table/6.6' # tariff barriers
url3 = 'http://wdi.worldbank.org/table/6.7' # trade facilitation
url4 = 'http://wdi.worldbank.org/table/6.8' # external debt
url5 = 'http://wdi.worldbank.org/table/6.11' # aid dependency
url6 = 'http://wdi.worldbank.org/table/6.14' # travel and tourism
url7 = 'http://wdi.worldbank.org/table/2.5' # unemployment
url8 = 'http://wdi.worldbank.org/table/4.10' # broader measure of national income
url9 = 'http://wdi.worldbank.org/table/4.16' # exchange rates and prices
# browser.visit(url)

In [4]:
# select the content from the direction of trade table (url1)
start1 = pd.read_html(url1)
trade_direction = start1[2]

trade_direction_list = trade_direction[0]
#trade_direction_list

In [5]:
# jsonify the trade_direction dataframe 
trade_direction.columns = ['country_name', 'toLow_withinRegion_2006', 'toLow_withinRegion_2016', 'toLow_outRegion_2006', 'toLow_outRegion_2016', 'toHi_withinRegion_2006', 'toHi_withinRegion_2016', 'fromLow_withinRegion_2006', 'fromLow_withinRegion_2016', 'fromLow_outRegion_2006', 'fromLow_outRegion_2016', 'fromHi_withinRegion_2006', 'fromHi_withinRegion_2016']
json_trade_direction = trade_direction.to_json("data/trade_direction.json", orient='index')
json_trade_direction

In [6]:
# select the content from the tariff barriers table (url2)
start2 = pd.read_html(url2)
tariff_barriers = start2[2]

#tariff_barriers.head()
# tariff_barriers_list = tariff_barriers[0]   # print out just the country names
# tariff_barriers_list                        # print out just the country names

In [7]:
# jsonify the trade_direction dataframe 
tariff_barriers.columns = ['country_name', 'year', 'AllProd_binding_coverage', 'AllProd_simpleMean_boundRate', 'AllProd_simpleMeanTariff', 'AllProd_weightedMeanTariff', 'AllProd_shareOf_tariffLines_INTLpeaks', 'AllProd_shareOf_tariffLines_SpecRates', 'PrimProd_simpleMeanTariff', 'PrimProd_weightedMeanTariff', 'ManufProd_simpleMeanTariff', 'ManufProd_weightedMeanTariff']
json_tariff_barriers = tariff_barriers.to_json("data/tariff_barriers.json", orient='index')
#json_tariff_barriers

In [8]:
# select the content from the trade faciliation table (url3)
start3 = pd.read_html(url3)
trade_facil = start3[2]

#trade_facil.head()

In [9]:
# jsonify the trade_facil dataframe 
trade_facil.columns = ['country_name', 'logistics_performance_index_1-5', 'customs_burden_index_1-7', 'leadTime_toExport_days', 'leadTime_toImport_days', 'cost_toExport', 'cost_toImport', 'shipping_connect_index_1-100', 'port_infrastructure_index_1-7']
json_trade_facil = trade_facil.to_json("data/trade_facil.json", orient='index')
# json_trade_facil

In [15]:
# select the content from the external debt table (url4)
start4 = pd.read_html(url4)
external_debt = start4[2]

external_debt.head()
# external_debt

,0,1,2,3,4,5,6,7,8,9,10
0,Afghanistan,2552,12.1,1982,0,285,11.2,..,4.0,8.2,125.6
1,Albania,9115,69.4,4028,2706,1892,20.8,..,10.4,27.4,78.2
2,Algeria,5699,3.4,1714,182,2096,36.8,..,0.6,0.8,3.5
3,American Samoa,..,..,..,..,..,..,..,..,..,..
4,Andorra,..,..,..,..,..,..,..,..,..,..


In [16]:
### I want to pull in the Iso3 codes and append them to this table
external_debt_countrycodes = pd.read_csv("world_codes.csv")
external_debt_countrycodes.head()

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [29]:
### Connect the two external debt tables
external_debt.columns = ['name', 'externalDebt_millions', 'externalDebt_%OfGNI', 'LTdebt_guaranteed', 'LTdebt_nonGuaranteed', 'STdebt_millions', 'STdebt_%OfTotalDebt', 'STdebt_%OfTotalReserves', 'totalDebtService_%OfExports', 'totalDebtService_%OfGNI', 'presentDebtValue_%OfExports']
external_debt_fixed = pd.concat([external_debt, external_debt_countrycodes], axis=1, join='inner')
external_debt_fixed.head()

,name,externalDebt_millions,externalDebt_%OfGNI,LTdebt_guaranteed,LTdebt_nonGuaranteed,STdebt_millions,STdebt_%OfTotalDebt,STdebt_%OfTotalReserves,totalDebtService_%OfExports,totalDebtService_%OfGNI,...,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,2552,12.1,1982,0,285,11.2,..,4.0,8.2,...,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Albania,9115,69.4,4028,2706,1892,20.8,..,10.4,27.4,...,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,NaN,150.0,154.0,NaN
2,Algeria,5699,3.4,1714,182,2096,36.8,..,0.6,0.8,...,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
3,American Samoa,..,..,..,..,..,..,..,..,..,...,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
4,Andorra,..,..,..,..,..,..,..,..,..,...,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN


In [11]:
# jsonify the external_debt dataframe 
external_debt.columns = ['country_name', 'externalDebt_millions', 'externalDebt_%OfGNI', 'LTdebt_guaranteed', 'LTdebt_nonGuaranteed', 'STdebt_millions', 'STdebt_%OfTotalDebt', 'STdebt_%OfTotalReserves', 'totalDebtService_%OfExports', 'totalDebtService_%OfGNI', 'presentDebtValue_%OfExports']
json_external_debt = external_debt.to_json("data/external_debt.json", orient='index')
#json_external_debt

In [45]:
# select the content from the aid dependency table (url5)
start5 = pd.read_html(url5)
aid_depend = start5[2]

aid_depend

,0,1,2,3,4,5,6,7,8
0,Afghanistan,4064,117,3641,489,20.6,117.9,53.9,..
1,Albania,169,59,137,76,1.4,5.7,3.0,5.9
2,Algeria,157,4,70,144,0.1,0.2,0.2,..
3,American Samoa,..,..,..,..,..,..,..,..
4,Andorra,..,..,..,..,..,..,..,..
5,Angola,207,7,168,30,0.2,2.6,0.7,1.2
6,Antigua and Barbuda,0,1,7,0,0.0,..,0.0,..
7,Argentina,3,0,35,35,0.0,0.0,0.0,0.0
8,Armenia,327,112,173,30,3.0,17.2,6.4,12.1
9,Aruba,..,..,..,..,..,..,..,..


In [41]:
### join this one with the iso3 country codes
aid_depend.columns = ['country_name', 'totalAid_millions', 'totalAid_perCapita', 'totalGrants_millions', 'TechCooperation_millions', 'NetDevAssist_%OfGNI', 'NetDevAssist_%ofGrossCapital', 'NetDevAssist_%ofImports', 'NetDevAssist_%ofGovExpenditures']
Wcode_aid_depend = aid_depend.merge(external_debt_countrycodes, left_on='country_name', right_on='name')
Wcode_aid_depend.head()

,country_name,totalAid_millions,totalAid_perCapita,totalGrants_millions,TechCooperation_millions,NetDevAssist_%OfGNI,NetDevAssist_%ofGrossCapital,NetDevAssist_%ofImports,NetDevAssist_%ofGovExpenditures,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code
0,Afghanistan,4064,117,3641,489,20.6,117.9,53.9,..,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Albania,169,59,137,76,1.4,5.7,3.0,5.9,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
2,Algeria,157,4,70,144,0.1,0.2,0.2,..,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
3,American Samoa,..,..,..,..,..,..,..,..,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN
4,Andorra,..,..,..,..,..,..,..,..,Andorra,AD,AND,20,ISO 3166-2:AD,Europe,Southern Europe,NaN,150.0,39.0,NaN


In [42]:
# jsonify the aid_dependency dataframe 
Wcode_aid_depend.columns = ['country_name', 'totalAid_millions', 'totalAid_perCapita', 'totalGrants_millions', 'TechCooperation_millions', 'NetDevAssist_%OfGNI', 'NetDevAssist_%ofGrossCapital', 'NetDevAssist_%ofImports', 'NetDevAssist_%ofGovExpenditures', 'drop1', 'drop2', 'ISO', 'drop3', 'drop4', 'drop5', 'drop6', 'drop7', 'drop8', 'drop9', 'drop10']
# json_aid_depend = aid_depend.to_json("data/aid_depend.json", orient='index')
# json_aid_depend
Wcode_aid_depend.head()

,country_name,totalAid_millions,totalAid_perCapita,totalGrants_millions,TechCooperation_millions,NetDevAssist_%OfGNI,NetDevAssist_%ofGrossCapital,NetDevAssist_%ofImports,NetDevAssist_%ofGovExpenditures,drop1,drop2,ISO,drop3,drop4,drop5,drop6,drop7,drop8,drop9,drop10
0,Afghanistan,4064,117,3641,489,20.6,117.9,53.9,..,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,NaN,142.0,34.0,NaN
1,Albania,169,59,137,76,1.4,5.7,3.0,5.9,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,NaN,150.0,39.0,NaN
2,Algeria,157,4,70,144,0.1,0.2,0.2,..,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,NaN,2.0,15.0,NaN
3,American Samoa,..,..,..,..,..,..,..,..,American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,NaN,9.0,61.0,NaN
4,Andorra,..,..,..,..,..,..,..,..,Andorra,AD,AND,20,ISO 3166-2:AD,Europe,Southern Europe,NaN,150.0,39.0,NaN


In [43]:
### drop the un-needed columns
final_aid_depend = Wcode_aid_depend.drop(['totalGrants_millions', 'TechCooperation_millions', 'NetDevAssist_%OfGNI', 'NetDevAssist_%ofGrossCapital', 'NetDevAssist_%ofImports', 'NetDevAssist_%ofGovExpenditures', 'drop1', 'drop2', 'drop3', 'drop4', 'drop5', 'drop6', 'drop7', 'drop8', 'drop9', 'drop10'], axis=1)
final_aid_depend

,country_name,totalAid_millions,totalAid_perCapita,ISO
0,Afghanistan,4064,117,AFG
1,Albania,169,59,ALB
2,Algeria,157,4,DZA
3,American Samoa,..,..,ASM
4,Andorra,..,..,AND
5,Angola,207,7,AGO
6,Antigua and Barbuda,0,1,ATG
7,Argentina,3,0,ARG
8,Armenia,327,112,ARM
9,Aruba,..,..,ABW


In [44]:
### convert the new aid_depend dataframe to json
final_aid_depend = final_aid_depend.to_json("data/final_aid_depend.json", orient='index')


In [14]:
# select the content from the travel and tourism table (url6)
start6 = pd.read_html(url6)
travel_tourism = start6[2]

# travel_tourism.head()

In [15]:
# jsonify the travel_tourism dataframe 
travel_tourism.columns = ['country_name', 'INTLtourists_inboundThous_2010', 'INTLtourists_inboundThous_2016', 'INTLtourists_outboundThous_2010', 'INTLtourists_outboundThous_2016', 'inboundTourismExpend_millions_2010', 'inboundTourismExpend_millions_2016', 'inboundTourismExpend_%ofExports_2010', 'inboundTourismExpend_%ofExports_2016', 'outboundTourismExpend_millions_2010', 'outboundTourismExpend_millions_2016', 'outboundTourismExpend_%ofExports_2010', 'outboundTourismExpend_%ofExports_2016']
json_travel_tourism = travel_tourism.to_json("data/travel_tourism.json", orient='index')
# json_travel_tourism

In [16]:
# select the content from the unemployment table (url7)
start7 = pd.read_html(url7)
unemployment = start7[2]

# unemployment.head()

In [22]:
# jsonify the unemployment dataframe 
unemployment.columns = ['country_name', 'unemployment_percentofMales_2000', 'unemployment_percentofMales_2016', 'unemployment_percentofFemales_2000', 'unemployment_percentofFemales_2016', 'youth_unemployment_percentofMales_2000', 'youth_unemployment_percentofMales_2016', 'youth_unemployment_percentofFemales_2000', 'youth_unemployment_percentofFemales_2016', 'unemployment_percentofTotal_baseEducation_2013-16', 'unemployment_percentofTotal_mediumEducation_2013-16', 'unemployment_percentofTotal_advancedEducation_2013-16']
json_unemployment = unemployment.to_json("data/unemployment.json", orient='index')
# json_unemployment


In [18]:
# select the content from the broad income measure table (url8)
start8 = pd.read_html(url8)
broadIncomeMeasure = start8[2]

# broadIncomeMeasure.head()

In [19]:
# jsonify the broad income measure dataframe 
broadIncomeMeasure.columns = ['country_name', 'GDP_billions_2016', 'GDP_%meanAnnualGrowth_2000-2016', 'GNI_billions_2016', 'GNI_%meanAnnualGrowth_2000-2016', 'comsumption_fixedCapital_%GNI_2016', 'naturalResourceDeplete_%ofGNI_2016', 'adjust_NNI_billions_2016', 'adjust_NNI_%meanAnnualGrowth_2000-2016']
json_broadIncomeMeasure = broadIncomeMeasure.to_json("data/broadIncomeMeasure.json", orient='index')
# json_broadIncomeMeasure

In [20]:
# select the content from the exchange rates table (url8)
start9 = pd.read_html(url9)
exchange_rates = start9[2]

# exchange_rates.head()

In [21]:
# jsonify the exchange rates dataframe 
exchange_rates.columns = ['country_name', 'DEC_altConversionFactor_2016', 'DEC_altConversionFactor_2017', 'PPP_convertFactor_2016', 'PPP_convertFactor_2017', 'ratio_PPP_toMarket_2016', 'ratio_PPP_toMarket_2017', 'effectiveExchangeRate_1-100_2016', 'effectiveExchangeRate_1-100_2017', 'GDP_implicitDeflator_%meanGrowth_2016', 'GDP_implicitDeflator_%meanGrowth_2017', 'CPI_%meanGrowth_2015-2016', 'CPI_%meanGrowth_2016-2017']
json_exchange_rates = exchange_rates.to_json("data/exchange_rates.json", orient='index')
# json_exchange_rates